# 🧮 Interactive Maths 
Game Matematika Sederhana dengan Hand Tracking

In [34]:
# Import library yang dibutuhkan
import cv2
import mediapipe as mp
import numpy as np
import os
import random

In [35]:
# Fungsi untuk generate soal matematika
def generate_math_problem():
    operators = ['+', '-', '*', '/']
    operator = random.choice(operators)
    
    if operator == '+':
        # Penjumlahan (hasil 0-10)
        result = random.randint(0, 10)
        num2 = random.randint(0, result)
        num1 = result - num2
    elif operator == '-':
        # Pengurangan (hasil 0-10)
        num1 = random.randint(0, 10)
        num2 = random.randint(0, num1)
    elif operator == '*':
        # Perkalian (hasil 0-10)
        result = random.randint(0, 10)
        factors = [i for i in range(1, 11) if result % i == 0]
        num1 = random.choice(factors)
        num2 = result // num1
    else:
        # Pembagian (hasil 0-10, tanpa sisa)
        num2 = random.randint(1, 10)
        num1 = random.randint(0, 10) * num2
    
    problem = f"{num1} {operator} {num2}"
    answer = eval(problem)
    return problem, float(answer)

# State untuk game
class GameState:
    def __init__(self):
        self.current_problem = None
        self.current_answer = None
        self.problems_count = 0
        self.correct_answers = 0
        self.total_problems = 5
        
    def new_problem(self):
        if self.problems_count < self.total_problems:
            self.current_problem, self.current_answer = generate_math_problem()
            return True
        return False
    
    def check_answer(self, user_answer):
        return abs(user_answer - self.current_answer) < 0.01

In [36]:
# Inisialisasi MediaPipe Hands dan Face Detection
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)

# Fungsi untuk mengecek apakah tangan berinteraksi dengan tombol
def check_button_interaction(hand_landmarks, button_pos, button_size):
    if hand_landmarks:
        # Menggunakan titik telunjuk (index finger tip)
        index_finger = hand_landmarks.landmark[8]
        x, y = int(index_finger.x * width), int(index_finger.y * height)
        
        # Cek apakah telunjuk berada di area tombol
        button_x, button_y = button_pos
        button_w, button_h = button_size
        
        if (button_x < x < button_x + button_w and 
            button_y < y < button_y + button_h):
            return True
    return False

# Fungsi untuk menempatkan gambar ke dalam frame
def overlay_image(background, overlay, position):
    h, w = overlay.shape[:2]
    y, x = position
    
    if overlay.shape[2] == 4:  # Jika ada alpha channel
        # Memisahkan alpha channel
        overlay_rgb = overlay[:,:,:3]
        alpha = overlay[:,:,3] / 255.0
        
        # Membuat alpha channel 3D
        alpha_3d = np.stack([alpha, alpha, alpha], axis=2)
        
        # Menghitung area yang akan ditimpa
        roi = background[y:y+h, x:x+w]
        # Menggabungkan gambar dengan alpha blending
        result = (overlay_rgb * alpha_3d + roi * (1 - alpha_3d))
        background[y:y+h, x:x+w] = result
    else:  # Jika tidak ada alpha channel
        background[y:y+h, x:x+w] = overlay
    return background

In [37]:
# Membaca gambar asset
logo_path = os.path.join(os.getcwd(), 'asset', 'logo.png')
button_path = os.path.join(os.getcwd(), 'asset', 'buttonstart.png')
bgcard_path = os.path.join(os.getcwd(), 'asset', 'backgroundcard.png')

# Membaca gambar
logo = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)
button = cv2.imread(button_path, cv2.IMREAD_UNCHANGED)
bgcard = cv2.imread(bgcard_path, cv2.IMREAD_UNCHANGED)

# Cek apakah gambar berhasil dibaca
if logo is None or button is None or bgcard is None:
    print("Error: Tidak dapat membaca file gambar!")
    print(f"Logo path: {logo_path}")
    print(f"Button path: {button_path}")
    print(f"Background Card path: {bgcard_path}")
else:
    # Mengubah ukuran gambar
    logo = cv2.resize(logo, (200, 200))
    button = cv2.resize(button, (200, 60))
    bgcard = cv2.resize(bgcard, (300, 150)) 

    # Inisialisasi webcam
    cap = cv2.VideoCapture(0)
    
    # State untuk mengontrol tampilan
    is_game_started = False
    game_state = GameState()  # Inisialisasi game state

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Gagal membaca frame dari webcam")
            break

        frame = cv2.flip(frame, 1)
        height, width = frame.shape[:2]

        # Konversi ke RGB untuk MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Deteksi tangan dan wajah
        hand_results = hands.process(rgb_frame)
        face_results = face_detection.process(rgb_frame)

        if not is_game_started:
            # Tampilan menu utama
            logo_x = (width - 200) // 2
            logo_y = 50
            button_x = (width - 200) // 2
            button_y = logo_y + 200 + 20

            frame = overlay_image(frame, logo, (logo_y, logo_x))
            frame = overlay_image(frame, button, (button_y, button_x))

            # Cek interaksi tangan dengan tombol
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    # Gambar landmark tangan
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS)
                    
                    # Cek interaksi dengan tombol start
                    if check_button_interaction(
                        hand_landmarks,
                        (button_x, button_y),
                        (200, 60)
                    ):
                        is_game_started = True
        else:
            # Mendeteksi posisi wajah untuk penempatan background card
            if face_results.detections:
                for detection in face_results.detections:
                    # Mendapatkan bounding box wajah
                    bbox = detection.location_data.relative_bounding_box
                    
                    # Konversi ke koordinat pixel
                    face_x = int(bbox.xmin * width)
                    face_y = int(bbox.ymin * height)
                    face_w = int(bbox.width * width)
                    face_h = int(bbox.height * height)
                    
                    # Menyesuaikan posisi background card di atas kepala
                    bgcard_x = face_x - (bgcard.shape[1] - face_w) // 2
                    bgcard_y = face_y - bgcard.shape[0] - 20  # 20 pixel di atas kepala
                    
                    # Memastikan background card tetap dalam frame
                    bgcard_x = max(0, min(bgcard_x, width - bgcard.shape[1]))
                    bgcard_y = max(0, min(bgcard_y, height - bgcard.shape[0]))
                    
                    # Menempelkan background card
                    frame = overlay_image(frame, bgcard, (bgcard_y, bgcard_x))

                    # Generate soal baru jika belum ada
                    if game_state.current_problem is None:
                        game_state.new_problem()

                    if game_state.current_problem:
                        # Konfigurasi text
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        font_scale = 1.0
                        thickness = 2
                        text = game_state.current_problem

                        # Hitung ukuran text
                        (text_width, text_height), baseline = cv2.getTextSize(
                            text, font, font_scale, thickness)

                        # Hitung posisi tengah dalam background card
                        text_x = bgcard_x + (bgcard.shape[1] - text_width) // 2
                        text_y = bgcard_y + (bgcard.shape[0] + text_height) // 2

                        # Tambahkan text dengan outline putih untuk keterbacaan
                        cv2.putText(frame, text, (text_x, text_y), font,
                                  font_scale, (255, 255, 255), thickness + 1)
                        cv2.putText(frame, text, (text_x, text_y), font,
                                  font_scale, (0, 0, 0), thickness)

        cv2.imshow('Interactive Maths', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    face_detection.close()
    cap.release()
    cv2.destroyAllWindows()